# load and pack using the schema

In [1]:
import numpy as np
from tensorflow.lite.tools import flatbuffer_utils
from tensorflow.lite.python import schema_py_generated as schema_fb

def load(path):
    return flatbuffer_utils.read_model_with_mutable_tensors(path)

def save(model , path):
    flatbuffer_utils.write_model(model, path)


def modify_conv2d_to_custom(model):
    # Create a new custom op code
    custom_op_code = schema_fb.OperatorCodeT()
    custom_op_code.customCode = "CONV_XOR"
    custom_op_code.builtinCode = schema_fb.BuiltinOperator.CUSTOM
    model.operatorCodes.append(custom_op_code)
    custom_op_index = len(model.operatorCodes) - 1

    for subgraph in model.subgraphs:
        for i ,operator in enumerate(subgraph.operators):
            op_code = model.operatorCodes[operator.opcodeIndex]
            if op_code.builtinCode == schema_fb.BuiltinOperator.CONV_2D and i>15:
                # Modify the operator to use your custom op
                operator.opcodeIndex = custom_op_index
                
                # Convert builtin options to custom options
                #to do

    return model



def pack_conv_weights(weights):
    """
    Pack a 4D numpy array of -1 and 1 values along the input channel dimension.
    
    Args:
    weights (numpy.ndarray): 4D array of shape (height, width, in_channels, out_channels)
    
    Returns:
    numpy.ndarray: Packed array with reduced in_channels dimension
    """
    assert set(np.unique(weights)) <= {-1, 1}, "Input array should only contain -1 and 1"
    
    in_channels,height, width, out_channels = weights.shape
    # Calculate the number of packed channels
    packed_channels = (in_channels + 31) // 32
    
    # Initialize the packed array
    packed_weights = np.zeros((packed_channels,height, width, out_channels), dtype=np.int32)
    
    for h in range(height):
        for w in range(width):
            for o in range(out_channels):
                for p in range(packed_channels):
                    for i in range(32):
                        if p * 32 + i < in_channels:
                            # Pack 32 values into a single integer
                            packed_weights[p, h, w, o] |= (
                                (weights[p * 32 + i, h, w, o] > 0).astype(int) << i
                            )
    
    return packed_weights

def modify_conv_weights(model):
    i=1
    for subgraph in model.subgraphs:
        for operator in subgraph.operators:
            op_code = model.operatorCodes[operator.opcodeIndex]
            if op_code.customCode == "CONV_XOR":
                # Assuming weights are the second input to the conv operator
                weight_tensor_index = operator.inputs[1]
                weight_tensor = subgraph.tensors[weight_tensor_index]
                new_name = f'conv_xor_w_{i}'
                weight_tensor.name=new_name
                i+=1
               # print(new_name.encode('utf-8'))
                buffer = model.buffers[weight_tensor.buffer]
                weights = np.frombuffer(buffer.data, dtype=np.float32)
                weights = weights.reshape(weight_tensor.shape)
               # print(type(weight_tensor.shape))
                weights=pack_conv_weights(weights)
                #print(type(np.array(weights.shape)))
                weight_tensor.shape = np.array(weights.shape)

                # # Modify data type
                weight_tensor.type = schema_fb.TensorType.INT32
                buffer.data = weights.tobytes()

    return model

    
def process_model(path):
    model=load(path)
    model=modify_conv2d_to_custom(model)
    model=modify_conv_weights(model)
    save(model,"dfsmn_xor.tflite")
  
        



In [2]:
process_model('dfsmn_bn.tflite')